In [5]:
import  jieba
import  re
import  string
stop_words_file = "G:/github/lesson7-8/stop_words/stop_words.utf8"
def cut(string):
    token =  list(jieba.cut(string))
    token = [word.strip() for word in token]
    return token
def get_stop_words_list():
    with open(stop_words_file,encoding="utf-8") as f:
       stop_words_list = f.readlines()
    return stop_words_list
#去除特殊符号
def remove_special_characters(text):
    tokens = cut(text)
    pattern = re.compile("[{0}\d+]".format(re.escape(string.punctuation)) )#print(string.punctuation) = !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
    filtered_tokens =  filter(None,[pattern.sub('', token) for token in tokens])
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text
#去停用词
def remove_stopwords(text):
    tokens = cut(text)
    stop_words_list = get_stop_words_list()
    filtered_tokens = [token for token in tokens if token not in stop_words_list] #这地方根本没有去停用词，如果去除了停用词，最后的评估模型的得分居然变低了。。。
    filtered_text = ''.join(filtered_tokens) #去掉空格
    return filtered_text


def normalize_corpus(corpus, tokenize=False):
    normalized_corpus = []
    #print(corpus)
    for text in corpus:
        text = remove_special_characters(text)
        text = remove_stopwords(text)
       # normalized_corpus.append(text)
        if tokenize:    #tokenize=True时执行
            text = ' '.join(cut(text))
        normalized_corpus.append(text)
        #print(text)
    #仅仅返回的是去掉特殊符号的文本
    return normalized_corpus

In [8]:
import os
import jieba
import warnings
from sklearn.feature_extraction.text import TfidfVectorizer
from  sklearn.feature_extraction.text import  CountVectorizer


from sklearn import metrics
warnings.filterwarnings('ignore')

def cut_words(file_path):
    text_with_spaces = ''
    text=open(file_path, 'r', encoding='gb18030').read()
    return text

def loadfile(file_dir, label):
    file_list = os.listdir(path + file_dir)
    words_list = []
    labels_list = []
    for file in file_list:
        file_path = path + file_dir + '/' + file
        words_list.append(cut_words(file_path))
        labels_list.append(label)
    words_list = normalize_corpus(words_list, True)
    return words_list, labels_list
def build_feature_matrix(documents, feature_type='frequency',
                         ngram_range=(1, 1), min_df=0.0, max_df=1.0):
    feature_type = feature_type.lower().strip()

    if feature_type == 'binary':
        vectorizer = CountVectorizer(binary=True,
                                     max_df=max_df, ngram_range=ngram_range)
    elif feature_type == 'frequency':
        vectorizer = CountVectorizer(binary=False, min_df=min_df,
                                     max_df=max_df, ngram_range=ngram_range)
    elif feature_type == 'tfidf':
        vectorizer = TfidfVectorizer(min_df=1,
                                 norm='l2',max_df =max_df,
                                 smooth_idf=True,
                                 use_idf=True,ngram_range=ngram_range)
    else:
        raise Exception("Wrong feature type entered. Possible values: 'binary', 'frequency', 'tfidf'")

    feature_matrix = vectorizer.fit_transform(documents).astype(float)

    return vectorizer, feature_matrix

from sklearn.cluster import KMeans
def k_means(feature_matrix, num_clusters=4):
    km = KMeans(n_clusters=num_clusters,
                max_iter=10000)
    km.fit(feature_matrix)
    clusters = km.labels_
    print("clusters=",clusters)
    return km, clusters

def get_cluster_data(clustering_obj, book_data,
                     feature_names, num_clusters,
                     topn_features=20):
    cluster_details = {}
    # 获取cluster的center
    ordered_centroids = clustering_obj.cluster_centers_.argsort()[:, ::-1]
    # 获取每个cluster的关键特征
    # 获取每个cluster的新闻
    for cluster_num in range(num_clusters):
        cluster_details[cluster_num] = {}
        cluster_details[cluster_num]['cluster_num'] = cluster_num
        key_features = [feature_names[index]
                        for index
                        in ordered_centroids[cluster_num, :topn_features]]
        cluster_details[cluster_num]['key_features'] = key_features
    return cluster_details


def print_cluster_data(cluster_data):
    # print cluster details
    for cluster_num, cluster_details in cluster_data.items():
        print('Cluster {} details:'.format(cluster_num))
        print('-' * 20)
        print('Key features:', cluster_details['key_features'])

if __name__ == "__main__":
    # 训练数据
    path = "G:/github/lesson7-8/data/"
    train_words_list1, train_labels1 = loadfile('train/女性', '0')

    train_words_list2, train_labels2 = loadfile('train/体育', '1')

    train_words_list3, train_labels3 = loadfile('train/文学', '2')

    train_words_list4, train_labels4 = loadfile('train/校园', '3')
    len_value = 260
    train_words_list = train_words_list1[:len_value] + train_words_list2[:len_value] + train_words_list3[:len_value] + train_words_list4[:len_value]

    train_labels = train_labels1 + train_labels2 + train_labels3 + train_labels4

    print("女性 len =" ,len(train_words_list1[:len_value]))
    print("体育 len =", len(train_words_list2[:len_value]))
    print("文学 len =", len(train_words_list3[:len_value]))
    print("校园 len =", len(train_words_list4[:len_value]))

    news_data = {}
    news_data["content"] = train_words_list
    news_data["type"] = train_labels
    stop_words = open(stop_words_file, 'r', encoding='utf-8').read()
    stop_words = stop_words.encode('utf-8').decode('utf-8-sig')  # 列表头部\ufeff处理
    stop_words = stop_words.split('\n')  # 根据分隔符分隔
    # 提取 tf-idf 特征
    vectorizer, train_features = build_feature_matrix(train_words_list,
                                                      feature_type='tfidf',
                                                      min_df=0.1, max_df=0.60,
                                                      ngram_range=(1, 1))
    # 查看特征数量
    print("shap2,",train_features.shape)
    # 获取特征名字
    feature_names = vectorizer.get_feature_names()
    # 打印某些特征
    print(feature_names[:10])
    num_clusters = 4
    km_obj, clusters = k_means(feature_matrix=train_features,
                               num_clusters=num_clusters)
    news_data['Cluster'] = clusters
    print("news_data['cluster']=", news_data['Cluster'])
    from collections import Counter

    # 获取每个cluster的数量
    c = Counter(clusters)
    print(c.items())
    print("news_data=")
    cluster_data = get_cluster_data(clustering_obj=km_obj,
                                    book_data=news_data,
                                    feature_names=feature_names,
                                    num_clusters=num_clusters,
                                    topn_features=20)

    print_cluster_data(cluster_data)

女性 len = 260
体育 len = 260
文学 len = 260
校园 len = 260
shap2, (1040, 11983)
['ac', 'act', 'arsenal', 'ata', 'a股', 'bb', 'bbb', 'boss', 'by', 'ca']
clusters= [0 2 2 ... 0 1 1]
news_data['cluster']= [0 2 2 ... 0 1 1]
dict_items([(0, 202), (2, 114), (1, 686), (3, 38)])
news_data=
Cluster 0 details:
--------------------
Key features: ['标签', '转发', '微博', '来自', '评论', '新浪', '上海', '今天', '周刊', '乔布斯', '星座', '体育', '出版', '时光', '第期', '中国', '英超', '直播', '皮皮', '狮子座']
Cluster 1 details:
--------------------
Key features: ['学校', '我们', '中国', '足球', '学生', '一个', '标签', '体育', '环保', '分享', '大学生', '可以', '校车', '腾讯', '女人', '皮肤', '自己', '校园', '学院', '没有']
Cluster 2 details:
--------------------
Key features: ['原文', '转发', '杂志', '评论', '微博', '面膜', '时尚', '意林', '推荐', '中国', '我们', '皮肤', '感恩', '时间', '好书', '女性', '可以', '支持', '粉丝', '新浪']
Cluster 3 details:
--------------------
Key features: ['nba', '排名', '腾讯', '直播', '中国', '巨星', '足球', '指数', '人均', '国际足球', '大帅府', '视频', '东北', '热烈欢迎', 'helloqqusersiveopenedthisaccounttobeclosertomychine

In [ ]:
#样本数量必须平衡，聚类才效果才稍微明显一些，感觉k-means聚类只是大致的分类，效果不是很好。